In [4]:
import sys
sys.path.append('../lib/')

import pandas as pd
import numpy as np
import geopandas as gpd

import workers
import v_ij
import math

from shapely.geometry import Point
from copy import deepcopy

### load grids, set area and average radius

In [5]:
grids = gpd.read_file('../results/grids_vgr_10km_density_deso.shp')

results_output = "../results/model_output_5km.txt"
results_benchmark = '../results/benchmark_5km.txt'

area = 25 # km

# A=pi*r_average^2
r_average = math.sqrt(area/math.pi) # km

In [6]:
print(grids)

          deso    density  zone  \
0    1435A0030   6.823680     1   
1    1435A0030   6.823680     2   
2    1486A0010   7.601277     3   
3    1486A0030   6.258427     4   
4    1421A0020   6.512613     5   
..         ...        ...   ...   
299  1473A0030   7.476964   300   
300  1447A0010   4.886978   301   
301  1446A0010   3.564339   302   
302  1446C1010  40.859174   303   
303  1446A0020   2.104448   304   

                                              geometry  
0    POLYGON ((270000.000 6490000.000, 270000.000 6...  
1    POLYGON ((270000.000 6510000.000, 270000.000 6...  
2    POLYGON ((270000.000 6530000.000, 270000.000 6...  
3    POLYGON ((270000.000 6540000.000, 270000.000 6...  
4    POLYGON ((280000.000 6440000.000, 280000.000 6...  
..                                                 ...  
299  POLYGON ((460000.000 6510000.000, 460000.000 6...  
300  POLYGON ((460000.000 6520000.000, 460000.000 6...  
301  POLYGON ((470000.000 6470000.000, 470000.000 6...  
302  POLY

### Get distance between grids

In [7]:
# This gives a stacked version
distances = workers.zone_distances(grids)
# This gives a matrix-style dataframe
df_d = distances.unstack(level=1)

Calculating distances between zones...


### Get population density and geometry

In [8]:
grid_name = list(grids.zone)
population_density = dict(zip(grids.zone, grids.density))
geometry = dict(zip(grids.zone, grids.geometry))

### Get $v^{tot}_{ij}$ between grids

In [93]:
# parameter = ln(f_max/f_min), f_min = 1/T, f_max = 1 T = 1000
T = 1000
f_max = 1
f_min = 1/T
parameter = math.log(f_max / f_min)

#ODM = {orig: {desti: v_{ori, desti}}}
# orig is the grid_name
# desti is the grid_name 


ODM_tot = dict()
for i in range(0, len(grid_name)):
    element = dict()
    for j in range(i + 1, len(grid_name)):
        number_of_trips = v_ij.average_daily_trips(population_density[grid_name[j]], area, r_average, df_d[grid_name[i]][grid_name[j]], parameter) + v_ij.average_daily_trips(population_density[grid_name[i]], area, r_average, df_d[grid_name[i]][grid_name[j]], parameter)
        element[grid_name[j]] = number_of_trips
    ODM_tot[grid_name[i]] = element

### Load Sweden VG zone data

In [9]:
zones = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
zones.loc[:, 'deso_3'] = zones.loc[:, 'deso'].apply(lambda x: x[:2])
zones_subset = zones.loc[zones['deso_3'] == '14', :]
zones_subset_info = dict(zip(zones_subset['deso'], zones_subset['geometry']))
zone_name = list(zones_subset['deso'])

### Aggregated Deso zone level

In [10]:
zones_subset.loc[:, 'deso_5'] = zones_subset.loc[:, 'deso'].apply(lambda x: x[:5])

D:\Anaconda\envs\geopandas_lab\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### Get  $v^{tot}_{ij}$ between aggregated Deso zones in VG

In [11]:
cover = []
checked = set()
for i in range(0, len(zone_name)):
    sub_cover = []
    for j in range(0, len(grid_name)):
        if j in checked:
            continue
        point_j = Point(geometry[grid_name[j]].centroid.x, geometry[grid_name[j]].centroid.y)
        if zones_subset_info[zone_name[i]].contains(point_j) == True:
            # grid_j in zone_i
            sub_cover.append(grid_name[j])
            checked.add(j)   # This grid has been occupied, we do not need to check it again.
    cover.append(sub_cover)
within = dict(zip(zone_name, cover))

In [12]:
bigzone_name = []
bigCover = []
subCover = []
old_name = zone_name[0][0:5]

bigzone_name.append(old_name)
subCover.extend(within[zone_name[0]])

for i in range(1, len(zone_name)):
    new_name = zone_name[i][0:5]

    if new_name == old_name:
        # this two zones belong the same big zone
        subCover.extend(within[zone_name[i]])

    if new_name != old_name:
        # find a new big zone
        #store old results
        bigCover.append(deepcopy(subCover))
        subCover.clear()

        #store new resutls
        bigzone_name.append(new_name)
        subCover.extend(within[zone_name[i]])

    old_name = new_name

# handle the lastest case
bigCover.append(subCover)


big_within = dict(zip(bigzone_name, bigCover))

In [13]:
print(big_within)

{'1401A': [103], '1401B': [], '1401C': [82], '1402C': [], '1407A': [], '1407C': [], '1415A': [69], '1415C': [51, 52], '1419A': [36], '1419B': [19], '1419C': [18], '1421A': [20, 5, 37, 21, 38], '1421B': [39], '1421C': [], '1427A': [8], '1427B': [9], '1427C': [], '1430A': [56, 57, 42, 58, 59], '1430C': [41], '1435A': [10, 25, 26, 1, 2, 11, 12, 13, 28, 27, 43, 44, 45], '1435B': [], '1435C': [], '1438A': [60, 61, 62, 77, 78, 79, 80], '1438C': [], '1439A': [73, 75, 76, 93, 94], '1439C': [74], '1440A': [66, 85, 107], '1440B': [], '1440C': [], '1441A': [84, 104, 105], '1441B': [], '1441C': [83], '1442A': [168, 169, 147, 148], '1442C': [], '1443A': [124, 146], '1443B': [], '1443C': [123], '1444A': [151], '1444C': [], '1445A': [149, 170, 150], '1445C': [], '1446A': [296, 298, 302, 299, 304], '1446B': [297], '1446C': [303], '1447A': [301, 259, 281, 282, 292, 293], '1447B': [], '1447C': [291], '1452A': [198, 199, 214, 215, 216, 229, 230], '1452B': [201], '1452C': [], '1460A': [95, 96, 81, 97, 98,

In [ ]:
ODM_big = dict()
for i in range(0, len(bigzone_name)):
    element = dict()
    for j in range(i, len(bigzone_name)):
        if i == j:
            average_daily_trips = 0
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(begin + 1, len(big_within[bigzone_name[i]])):
                    average_daily_trips = average_daily_trips + ODM_tot[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[i]][end]]

            element[bigzone_name[j]] = 2 * average_daily_trips
        if i != j:
            average_daily_trips = 0
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[j]])):
                    average_daily_trips = average_daily_trips + ODM_tot[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]]
            element[bigzone_name[j]] = average_daily_trips

    ODM_big[bigzone_name[i]] = element

### From upper triangular matrix to a vector

In [ ]:
model_output = np.zeros(len(bigzone_name)*len(bigzone_name))   # the vector of model_output is stored here
for i in range(0, len(bigzone_name)):
    for j in range(0, len(bigzone_name)):
        if i <= j:
            model_output[i + j * len(bigzone_name)] = ODM_big[bigzone_name[i]][bigzone_name[j]]
        if i > j:
            model_output[i + j * len(bigzone_name)] = ODM_big[bigzone_name[j]][bigzone_name[i]]

In [ ]:
np.savetxt(results_output, model_output)

#--------------From here we prepare the groundtruth data----------#

In [ ]:
zones = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
zones = zones.rename(columns={"deso": "zone"})[['zone', 'geometry']]


trips = pd.read_csv("../dbs/sweden/survey/day_trips.csv")
trips = trips.loc[:, ["sub_id", 'trip_id', 'trip_main_id', 'distance_main',
                              'date', "origin_main_deso", "desti_main_deso", 'trip_weight']]
trips = trips.drop_duplicates(subset=["sub_id", 'trip_id', 'trip_main_id'])
trips["T"] = trips["date"].apply(lambda x: pd.to_datetime(x))
trips = trips.loc[~trips["T"].apply(lambda x: x.weekday()).isin([5, 6]), :]
trips.dropna(axis=0, how='any', inplace=True)

odms = trips.groupby(['origin_main_deso', 'desti_main_deso']).sum()['trip_weight']
odms = odms.reindex(pd.MultiIndex.from_product([zones.zone, zones.zone], names=['ozone', 'dzone']), fill_value=0)

In [ ]:
ODM = dict()
for i in range(0, len(zone_name)):
    element = dict()
    for j in range(0, len(zone_name)):
        element[zone_name[j]] = odms.at[zone_name[i], zone_name[j]]
    ODM[zone_name[i]] = element

In [ ]:
bigzone_name = []
bigCover = []
subCover = []
old_name = zone_name[0][0:5]

bigzone_name.append(old_name)
subCover.append(zone_name[0])

for i in range(1, len(zone_name)):
    new_name = zone_name[i][0:5]

    if new_name == old_name:
        # this two zones belong the same big zone
        subCover.append(zone_name[i])

    if new_name != old_name:
        # find a new big zone
        #store old results
        bigCover.append(deepcopy(subCover))
        subCover.clear()

        #store new resutls
        bigzone_name.append(new_name)
        subCover.append(zone_name[i])

    old_name = new_name

# handle the lastest case
bigCover.append(subCover)


big_within = dict(zip(bigzone_name, bigCover))

### Aggregated groundtruth data

In [ ]:
ODM_truth = dict()
for i in range(0, len(bigzone_name)):
    element = dict()
    for j in range(0, len(bigzone_name)):
        average_daily_trips = 0
        if i == j:
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[i]])):
                    average_daily_trips = average_daily_trips + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[i]][end]]
                    
        if i != j:
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[j]])):
                    average_daily_trips = average_daily_trips + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]]
        
        element[bigzone_name[j]] = average_daily_trips

    ODM_truth[bigzone_name[i]] = element

In [ ]:
benchmark = np.zeros(len(bigzone_name)*len(bigzone_name))   # the vector of groundtruth data is stored here
for i in range(0, len(bigzone_name)):
    for j in range(0, len(bigzone_name)):
            benchmark[i + j * len(bigzone_name)] = ODM_truth[bigzone_name[i]][bigzone_name[j]]

In [ ]:
np.savetxt(results_benchmark, benchmark)